In [246]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf
from prophet import Prophet
import timesfm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# --------------------------
# Required Packages for Models
# --------------------------
import xgboost as xgb
from lightgbm import LGBMRegressor

# PyTorch and related libraries
import torch
import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

import logging
cmdstanpy_logger = logging.getLogger("cmdstanpy")
absl_logger = logging.getLogger("absl")
cmdstanpy_logger.disabled = True
absl_logger.disabled = True

In [247]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 44779.05it/s]


### Load data

In [248]:
data = pd.read_csv('../../data/df_merged.csv')

In [249]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712264 entries, 0 to 712263
Data columns (total 43 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Codigo                  712264 non-null  object 
 1   Descripcion             703248 non-null  object 
 2   fechaHora               703248 non-null  object 
 3   PrecEuro                703248 non-null  float64
 4   Energia                 703248 non-null  float64
 5   date                    703248 non-null  object 
 6   hour                    703248 non-null  float64
 7   week                    703248 non-null  float64
 8   day_of_week             703248 non-null  float64
 9   month                   703248 non-null  float64
 10  day_of_month            703248 non-null  float64
 11  is_weekend              712264 non-null  int64  
 12  hour_sin                703248 non-null  float64
 13  hour_cos                703248 non-null  float64
 14  dow_sin             

### Ordinal Encoding

In [250]:
encoder = OrdinalEncoder()
cat_cols=data.select_dtypes(include=['object']).columns
cat_cols = cat_cols.drop('fechaHora')
data[cat_cols] = encoder.fit_transform(data[cat_cols])

### Data Preparation

In [251]:
data = data.sort_index()

# Create a log-transformed and first-differenced (stationary) series.
data['Energia_log'] = np.log(data['Energia'] + 1).shift(24 * 30)  # add a small constant so log(0+1)=0
data['PrecEuro'] = data['PrecEuro'].shift(24 * 30)

data['Energia_stationary'] = data['Energia_log'].diff()

n_lags = 24
for lag in range(1, n_lags + 1):
    data[f'lag_{lag}'] = data['Energia_stationary'].shift(lag)

data = data.dropna()

In [252]:
EXCLUDED_COLS = ['Energia_stationary', 'Energia_log', 'Energia', 'fechaHora']
FEATURES = [col for col in data.columns if col not in EXCLUDED_COLS]
TARGET = 'Energia_stationary'

## Models

### TimesFM

In [253]:
class TimesFMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TimesFMModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [254]:
def prepare_timesfm(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame) -> tuple:
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    return X_train_tensor, y_train_tensor, X_test_tensor

In [255]:
TIMESFM_MODEL_PARAMS = {
    'hidden_dim': 100,
    'output_dim': 1,
}

TIMESFM_TRAIN_PARAMS = {
    'lr': 0.01,
}


In [256]:
def train_timesfm(
    X_train_tensor: torch.Tensor, 
    y_train_tensor: torch.Tensor,
    TIMESFM_MODEL_PARAMS: dict,
    TIMESFM_TRAIN_PARAMS: dict,
    epochs: int = 150
    ) -> TimesFMModel:

    timesfm_model = TimesFMModel(input_dim=len(FEATURES), **TIMESFM_MODEL_PARAMS)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(timesfm_model.parameters(), **TIMESFM_TRAIN_PARAMS)
    
    # Train the model.
    timesfm_model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = timesfm_model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
    return timesfm_model
    
def evaluate_timesfm(timesfm_model: TimesFMModel, X_test_tensor: torch.Tensor) -> np.ndarray:
    timesfm_model.eval()
    with torch.no_grad():
        y_pred_timesfm = timesfm_model(X_test_tensor).numpy().flatten()
        
    return y_pred_timesfm

### LSTM

In [257]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # batch_first=True makes input shape (batch, seq, feature)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden and cell states with zeros (and send to same device as x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        # Use the output from the final time step
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [258]:
def prepare_lstm(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame) -> tuple:
    selected_features = [
        'lag1_Energia', 'lag24_Energia', 'roll24_mean_Energia',
        'hour', 'day_of_week', 'day_of_month', 'month', 'is_weekend',
        'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
        'PrecEuro', 'cum_energy'
    ]
    X_train_selected = X_train[selected_features].values  # shape: (num_samples, num_features)
    X_test_selected  = X_test[selected_features].values

    # Add a time dimension (sequence length = 1)
    X_train_lstm = torch.tensor(X_train_selected, dtype=torch.float32).unsqueeze(1)  # shape: (samples, 1, num_features)
    X_test_lstm  = torch.tensor(X_test_selected, dtype=torch.float32).unsqueeze(1)
    y_train_lstm = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    
    return X_train_lstm, y_train_lstm, X_test_lstm

In [259]:
LSTM_MODEL_PARAMS = {
    'hidden_size': 50,
    'num_layers': 1,
    'output_size': 1
}

LSTM_TRAIN_PARAMS = {
    'lr': 0.01,
}


In [ ]:
def train_lstm(
    X_train: torch.Tensor, 
    y_train: torch.Tensor, 
    LSTM_MODEL_PARAMS: dict,
    LSTM_TRAIN_PARAMS: dict,
    epochs: int=150
    ) -> LSTMModel:
    
    input_size = X_train.shape[2]
    lstm_model = LSTMModel(input_size, **LSTM_MODEL_PARAMS)

    criterion_lstm = nn.MSELoss()
    optimizer_lstm = optim.Adam(lstm_model.parameters(), **LSTM_TRAIN_PARAMS)
    
    lstm_model.train()
    for epoch in range(epochs):
        optimizer_lstm.zero_grad()
        outputs = lstm_model(X_train)
        loss = criterion_lstm(outputs, y_train)
        loss.backward()
        optimizer_lstm.step()
        
    return lstm_model
    
    
def evaluate_lstm(lstm_model: LSTMModel, X_test_lstm: torch.Tensor) -> np.ndarray:
    lstm_model.eval()
    with torch.no_grad():
        y_pred_lstm = lstm_model(X_test_lstm).numpy().flatten()
    return y_pred_lstm

### XGB

In [261]:
XGB_PARAMS = {
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.01,
    'verbosity': 0
}

In [262]:
def train_xgb(
    X_train: pd.DataFrame, 
    y_train: pd.Series, 
    X_test: pd.DataFrame,
    y_test: pd.Series,
    XGB_PARAMS: dict) -> xgb.XGBRegressor:    
    xgb_reg = xgb.XGBRegressor(**XGB_PARAMS)
    xgb_reg.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=False
    )
    return xgb_reg
    
def evaluate_xgb(xgb_reg: xgb.XGBRegressor, X_test: pd.DataFrame) -> np.ndarray:
    return xgb_reg.predict(X_test)
    

### LGBM

In [263]:
LGBM_PARAMS = {
    'n_estimators': 1000,
    'learning_rate': 0.01,
    'max_depth': 3
}

In [264]:
def train_lgbm(
    X_train: pd.DataFrame, 
    y_train: pd.Series, 
    X_test: pd.DataFrame,
    y_test: pd.Series,
    LGBM_PARAMS: dict
    ) -> LGBMRegressor:    
    lgbm_reg = LGBMRegressor(**LGBM_PARAMS)
    lgbm_reg.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)]
    )
    return lgbm_reg
    
def evaluate_lgbm(lgbm_reg: LGBMRegressor, X_test: pd.DataFrame) -> np.ndarray:
    return lgbm_reg.predict(X_test)

### Prophet

In [265]:
def prepare_prophet(train: pd.DataFrame) -> pd.DataFrame:
    train_prophet = train.reset_index()
    train_prophet['ds'] = pd.to_datetime(train_prophet['fechaHora'])

    train_prophet = train_prophet.rename(columns={TARGET: 'y'})

    return train_prophet[['ds', 'y', 'PrecEuro', 'cum_energy']]

def train_prophet(train_prophet: pd.DataFrame) -> Prophet:

    prophet_model = Prophet()
    prophet_model.add_regressor('PrecEuro')
    prophet_model.add_regressor('cum_energy')

    # Fit the model
    prophet_model.fit(train_prophet)
    
    return prophet_model

def evaluate_prophet(prophet_model: Prophet, test: pd.DataFrame) -> np.ndarray:

    test_prophet = test.reset_index()
    test_prophet['ds'] = pd.to_datetime(test_prophet['fechaHora'])
    future = test_prophet[['ds', 'PrecEuro', 'cum_energy']]

    # Forecast
    forecast = prophet_model.predict(future)
    return forecast['yhat'].values

## Benchmark

In [266]:
tss = TimeSeriesSplit(n_splits=5, test_size=24 * 30, gap=24)

model_names = ['TimesFM', 'XGB', 'LGBM', 'Prophet', 'LSTM']
results = {model: {'mae': [], 'mape': []} for model in model_names}

In [267]:
def mae_score(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def mape_score(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.abs(y_true+ 1e-3))) * 100

In [ ]:
fold_num = 0
for train_idx, test_idx in tss.split(data):
    print(f"\n===== Fold {fold_num} =====")
    
    # Split the data into train and test folds
    train = data.iloc[train_idx]
    test = data.iloc[test_idx]
    
    # Common splits for models that require X and y inputs:
    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test  = test[FEATURES]
    y_test  = test[TARGET]
    
    # --------------------------
    if 'Prophet' in model_names:
        data_prophet = prepare_prophet(train)
        prophet_model = train_prophet(data_prophet)
        y_pred_prophet = evaluate_prophet(prophet_model, test)
        
        mae_prophet = mae_score(y_test, y_pred_prophet)
        mape_prophet = mape_score(y_test, y_pred_prophet)
        
        results['Prophet']['mae'].append(mae_score(y_test, y_pred_prophet))
        results['Prophet']['mape'].append(mape_prophet)
        
        print(f"Prophet    --> MAE: {mae_prophet:.4f}, MAPE: {mape_prophet:.4f}")
    
    # --------------------------
    if 'XGB' in model_names:
        xgb_model = train_xgb(X_train, y_train, X_test, y_test, XGB_PARAMS)
        y_pred_xgb = evaluate_xgb(xgb_model, X_test)
        
        mae_xgb = mae_score(y_test, y_pred_xgb)
        mape_xgb = mape_score(y_test, y_pred_xgb)
        
        results['XGB']['mae'].append(mae_xgb)
        results['XGB']['mape'].append(mape_xgb)
        
        print(f"XGBoost    --> MAE: {mae_xgb:.4f}, MAPE: {mape_xgb:.4f}")
    
    # --------------------------
    if 'LGBM' in model_names:
        lgbm_model = train_lgbm(X_train, y_train,  X_test, y_test, LGBM_PARAMS)
        y_pred_lgbm = evaluate_lgbm(lgbm_model, X_test)

        mae_lgbm = mae_score(y_test, y_pred_lgbm)
        mape_lgbm = mape_score(y_test, y_pred_lgbm)

        results['LGBM']['mae'].append(mae_lgbm)
        results['LGBM']['mape'].append(mape_lgbm)

        print(f"LightGBM   --> MAE: {mae_lgbm:.4f}, MAPE: {mape_lgbm:.4f}")

    # --------------------------
    if 'TimesFM' in model_names:
        X_train_timesfm, y_train_timesfm, X_test_timesfm = prepare_timesfm(X_train, y_train, X_test)  
        timesfm_model = train_timesfm(X_train_timesfm, y_train_timesfm, TIMESFM_MODEL_PARAMS, TIMESFM_TRAIN_PARAMS)
        y_pred_timesfm = evaluate_timesfm(timesfm_model, X_test_timesfm)
        
        mae_timesfm = mae_score(y_test, y_pred_timesfm)
        mape_timesfm = mape_score(y_test, y_pred_timesfm)
        
        results['TimesFM']['mae'].append(mae_timesfm)
        results['TimesFM']['mape'].append(mape_timesfm)
        
        print(f"TimesFM   --> MAE: {mae_timesfm:.4f}, MAPE: {mape_timesfm:.4f}")
    
    # --------------------------
    # LSTM Model (PyTorch)
    # --------------------------
    if 'LSTM' in model_names:
        X_train_lstm, y_train_lstm, X_test_timesfm = prepare_lstm(X_train, y_train, X_test)
        lstm_model = train_lstm(X_train_lstm, y_train_lstm, LSTM_MODEL_PARAMS, LSTM_TRAIN_PARAMS)
        y_pred_lstm = evaluate_lstm(lstm_model, X_test_timesfm)
        
        mae_lstm = mae_score(y_test, y_pred_lstm)
        mape_lstm = mape_score(y_test, y_pred_lstm)
        
        results['LSTM']['mae'].append(mae_lstm)
        results['LSTM']['mape'].append(mape_lstm)
        
        print(f"LSTM       --> MAE: {mae_lstm:.4f}, MAPE: {mape_lstm:.4f}")
    
    fold_num += 1

# --------------------------
# STEP 5: Aggregate and Display Results
# --------------------------
benchmark_results = []
for model in model_names:
    avg_mae = np.mean(results[model]['mae'])
    avg_mape = np.mean(results[model]['mape'])
    benchmark_results.append({
        'Model': model,
        'MAE': avg_mae,
        'MAPE': avg_mape
    })

results_df = pd.DataFrame(benchmark_results)
print("\n===== Benchmark Results =====")
print(results_df)


===== Fold 0 =====


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Prophet    --> MAE: 0.2080, MAPE: 2618.8593
XGBoost    --> MAE: 0.0865, MAPE: 200.4457
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016265 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11641
[LightGBM] [Info] Number of data points in the train set: 393383, number of used features: 62
[LightGBM] [Info] Start training from score 0.000038
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Prophet    --> MAE: 0.2480, MAPE: 2266.6483
XGBoost    --> MAE: 0.1301, MAPE: 165.3583
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015660 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11643
[LightGBM] [Info] Number of data points in the train set: 394103, number of used features: 62
[LightGBM] [Info] Start training from score 0.000039
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
